# 0. Import Libraries and Test Camera

In [25]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import uuid
import time

print(os.name)

nt


In [26]:
capture = cv2.VideoCapture(0)

while True:
    isTrue, frame = capture.read()
    cv2.imshow('Video', frame)
    if cv2.waitKey(20) & 0xFF==ord('d'):
        break

capture.release()
cv2.destroyAllWindows()
    

# 1. Setup Folders

In [27]:
DATADIR = os.path.join('Datasets', 'SortedImages')
CATEGORIES = ["empty","black_piece", "white_piece"]
COLLECT_IMAGES_PATH = os.path.join('Datasets', 'CollectedImages')
IMG_SIZE=70
number_imgs = 1

if not os.path.exists(COLLECT_IMAGES_PATH):
    !mkdir {COLLECT_IMAGES_PATH}
    
for category in CATEGORIES:
    IMG_PATH = os.path.join(DATADIR, category)
    if not os.path.exists(IMG_PATH):
        !mkdir {IMG_PATH}

# 2. Capture Images

## Calibrate by finding corners of empty board

In [28]:

###############################################
HEIGHT = 560
WIDTH = 560
#################################################


#--------------------------HELPERS----------------------------------------------
def preprocess(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_blur = cv2.GaussianBlur(img_gray, (3,3),1)
    img_threshold = cv2.adaptiveThreshold(img_blur, 255,1,1,11,2)
    return img_threshold

def reorder(pts):
    pts = pts.reshape((4,2))
    new_pts = np.zeros((4,1,2))
    sum = pts.sum(1)
    new_pts[0] = pts[np.argmin(sum)]
    new_pts[3] = pts[np.argmax(sum)]
    diff = np.diff(pts,axis=1)
    new_pts[1] = pts[np.argmin(diff)]
    new_pts[2] = pts[np.argmax(diff)]
    return new_pts

def findBiggestContour(contours):
    pts = np.array([])
    max_area = 0
    for i in contours:
        area = cv2.contourArea(i)
        if area > 50:
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i,0.02*peri,True)
            if area > max_area and len(approx) == 4:
                pts = approx
                max_area = area
    return reorder(pts), max_area

def find_board_corners(img):
    processed_img = preprocess(img)
    contours, heirarchy = cv2.findContours(processed_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    biggest, maxArea = findBiggestContour(contours)
    return biggest
    
def crop_board(img, corners):
    original_img = np.float32(corners)
    new_img = np.float32([[0, 0], [WIDTH, 0], [0, HEIGHT], [WIDTH, HEIGHT]])
    matrix = cv2.getPerspectiveTransform(original_img, new_img)
    imgWarpColored = cv2.warpPerspective(frame, matrix, (WIDTH, HEIGHT))
    return imgWarpColored

def get_cells(img):
    rows = np.vsplit(img,8)
    cells = [[],[],[],[],[],[],[],[]]
    for r in range(8):
        cols = np.hsplit(rows[r], 8)
        for c in cols:
            cells[r].append(c)
    return cells


In [29]:
#calibrate

cap = cv2.VideoCapture(0)

ret,calibrate_img = cap.read()
corners = find_board_corners(calibrate_img)

### Test Camera

In [30]:
capture = cv2.VideoCapture(0)

while True:
    isTrue, frame = capture.read()
    cv2.imshow('Video', crop_board(frame, corners))
    if cv2.waitKey(20) & 0xFF==ord('d'):
        break

capture.release()
cv2.destroyAllWindows()

## Capture Images

In [174]:
cap = cv2.VideoCapture(0)
for imgnum in range(number_imgs):
    print('Collecting image {}'.format(imgnum))
    ret, frame = cap.read()
    frame = crop_board(frame, corners)
    cv2.imshow('frame', frame)
    cells = get_cells(frame)
    for i in range(8):
        for j in range(8):
            imgname = os.path.join(COLLECT_IMAGES_PATH,'img.'+'{}.jpg'.format(str(uuid.uuid1())))
            cv2.imwrite(imgname, cells[i][j])
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

## Data Augmentation

In [27]:
from keras.preprocessing.image import ImageDataGenerator
from skimage import io
datagen = ImageDataGenerator(        
        vertical_flip = True,
        horizontal_flip = True,
        brightness_range = (0.5, 1.5))
import numpy as np
import os
from PIL import Image
dataset = []
AUGDIR = os.path.join('Datasets', 'AugmentedImages')
DATADIR = os.path.join('Datasets', 'SortedImages')
for category in CATEGORIES:
    dataset = []
    my_images = os.path.join(DATADIR, category)
    for image_name in os.listdir(my_images):
        image = io.imread(os.path.join(my_images,image_name))        
        image = Image.fromarray(image, 'RGB')        
        image = image.resize((IMG_SIZE,IMG_SIZE)) 
        dataset.append(np.array(image))
    x = np.array(dataset)
    
    save_dir = os.path.join(AUGDIR, category)
    if not os.path.exists(save_dir):
        !mkdir {save_dir}
    i = 0
    for batch in datagen.flow(x, batch_size=5,
                              save_to_dir= save_dir,
                              save_prefix='dr',
                              save_format='jpg'):    
        i += 1    
        if i > 400:        
            break


# 3. Read the Images

In [7]:
from keras.preprocessing.image import ImageDataGenerator

In [8]:
#At this point, move the collected images in the corresponding folders
IMG_SIZE = 70

In [9]:
training_data = []
AUGDIR = os.path.join('Datasets', 'AugmentedImages')
def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exeption as e:
                pass

create_training_data()

In [10]:
import random

random.shuffle(training_data)

In [11]:
X = []
y = []


In [12]:
for features, label in training_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y = np.array(y)

In [13]:
import pickle

pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()


# 4. Train Model

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time
from tensorflow import keras
from tensorflow.keras import layers

In [15]:
NAME = "ChessClassification-CNN-{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

In [16]:
data_augmentation = keras.Sequential([
    layers.experimental.preprocessing.RandomRotation(0.3),
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(IMG_SIZE,IMG_SIZE,3)),
    layers.experimental.preprocessing.RandomFlip("vertical", input_shape=(IMG_SIZE,IMG_SIZE,3)),
    layers.experimental.preprocessing.RandomContrast(0.3),
])

In [17]:
X = X/255.0

model = Sequential()

#model.add(data_augmentation)
model.add(Conv2D(64, (3,3), input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.2))

model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ['accuracy'])

model.fit(X, y, batch_size = 32, validation_split = 0.2, epochs = 12, callbacks = [tensorboard])

model.save('my_chess_model')

Epoch 1/12
55/55 [==============================] - 14s 221ms/step - loss: 0.6089 - accuracy: 0.7184 - val_loss: 0.1345 - val_accuracy: 0.9909
Epoch 2/12
55/55 [==============================] - 10s 189ms/step - loss: 0.1285 - accuracy: 0.9681 - val_loss: 0.0558 - val_accuracy: 0.9727
Epoch 3/12
55/55 [==============================] - 10s 188ms/step - loss: 0.0590 - accuracy: 0.9886 - val_loss: 0.0223 - val_accuracy: 0.9954
Epoch 4/12
55/55 [==============================] - 10s 190ms/step - loss: 0.0512 - accuracy: 0.9914 - val_loss: 0.0128 - val_accuracy: 1.0000
Epoch 5/12
55/55 [==============================] - 10s 190ms/step - loss: 0.0400 - accuracy: 0.9903 - val_loss: 0.0054 - val_accuracy: 1.0000
Epoch 6/12
55/55 [==============================] - 11s 193ms/step - loss: 0.0275 - accuracy: 0.9920 - val_loss: 0.0043 - val_accuracy: 0.9977
Epoch 7/12
55/55 [==============================] - 11s 191ms/step - loss: 0.0218 - accuracy: 0.9949 - val_loss: 0.0087 - val_accuracy: 0.9977

# 5. Make Predictions

In [31]:
IMG_SIZE=70

In [32]:
img = 'pred2.jpg'

def prepare(filepath):
    img_array = cv2.imread(filepath)
    img_array = cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
    return img_array.reshape(-1,IMG_SIZE,IMG_SIZE,3)

model = tf.keras.models.load_model('my_chess_model')

# cv2.imshow('daefadf',im)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

prediction = model.predict([prepare(img)])

print(prediction.argmax())

1


In [34]:
capture = cv2.VideoCapture(0)

ret,frame = capture.read()

img = crop_board(frame,corners)
cells = get_cells(img)

board = [[None for i in range(8)] for j in range(8)]

for i in range(8):
    for j in range(8):
        cells[i][j] = cv2.resize(cells[i][j], (IMG_SIZE,IMG_SIZE))
        prediction = model.predict([cells[i][j].reshape(-1,IMG_SIZE,IMG_SIZE,3)])
        board[i][j] = prediction.argmax()

print(board)
        


[[2, 2, 0, 0, 0, 0, 1, 1], [2, 2, 0, 0, 0, 0, 1, 1], [2, 2, 0, 0, 0, 0, 2, 1], [2, 0, 2, 0, 0, 0, 0, 1], [2, 0, 0, 2, 1, 0, 0, 1], [2, 2, 0, 0, 0, 0, 1, 1], [2, 2, 0, 0, 0, 0, 1, 1], [2, 2, 0, 0, 0, 0, 1, 1]]
